In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import palantir

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, frameon=False)  # low dpi (dots per inch) yields small inline figures
sys.path.insert(0,'/home/users/jjzhu/source_code/aloe/src')
sys.path.insert(0,'./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
scanpy==1.4.4.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.1 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


In [40]:
from liver_info import load_processsed_hepatocyte_data

dat_dir = "/share/PI/sabatti/sc_data/liver2k"
res_dir = "/share/PI/sabatti/feat_viz/real_analysis_result/analysis_060719"
main_dir = os.path.join(dat_dir, "analysis_on_data_original")
x_k, x_d, obs_df, var_df = load_processsed_hepatocyte_data(main_dir, center=False, scale=False)
norm_df = pd.DataFrame(x_d, columns=var_df.gene_ids, index=obs_df.index)

In [41]:
from liver_info import load_zonation_result
zone_vals = load_zonation_result(dat_dir, just_vals=True)
idx = [i for i, v in enumerate(zone_vals) if v ==1]
start_cell = obs_df.iloc[idx[0]].name

In [42]:
from shared_utils import run_palantir_pseudotime
pr_res, out_df = run_palantir_pseudotime(norm_df, start_cell)

Determing nearest neighbor graph...
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
Sampling and flocking waypoints...
Time for determining waypoints: 0.03228768110275269 minutes
Determining pseudotime...
Shortest path distances using 30-nearest neighbor graph...
Time for shortest paths: 0.3087550719579061 minutes
Iteratively refining the pseudotime...
Correlation at iteration 1: 1.0000
Entropy and branch probabilities...
Markov chain construction...
Identification of terminal states...
Computing fundamental matrix and absorption probabilities...
Project results to all cells...


In [45]:
# corr_df = df.corr('spearman')
out_df['zone'] = zone_vals
out_df['cell_id'] = out_df.index
df = out_df.drop(['entropy'], axis=1)
df.corr('spearman')

,pseudotime,zone
pseudotime,1.000000,0.771033
zone,0.771033,1.000000


In [46]:
out_dir = '/share/PI/sabatti/feat_viz/real_analysis_result/analysis_060719/hepa_data'
fn = os.path.join(out_dir, 'ti_methods', 'palantir_obs_df.csv')
out_df.to_csv(fn, index=False)
print('Saved {}'.format(fn))

Saved /share/PI/sabatti/feat_viz/real_analysis_result/analysis_060719/hepa_data/ti_methods/palantir_obs_df.csv
